## 2.7 Keras Project (IMAGE): Transfer Learning

### Project Flow:

1. Download/Load Pre-trained models and data
2. Build CNN model from data directly
3. Transfer Learning with VGG16
4. Transfer Learning with Xception
5. Make prediction

In [36]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

import cv2                
import matplotlib.pyplot as plt                        
%matplotlib inline      

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image                  
from tqdm import tqdm
from keras.applications.resnet50 import preprocess_input, decode_predictions

from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True      

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint  

data_path='../ClassSampleData/'

In [37]:
# Pre-trained model
# Human detector
face_cascade = cv2.CascadeClassifier(data_path+'haarcascade_frontalface_alt.xml')
# Dog detector
ResNet50_model = ResNet50(weights='imagenet')

In [38]:
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

def face_detector(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    return len(faces) > 0

# convert image to 4D tensor (obs, row, column, channels) for dog_detector

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

def dog_detector(img_path):
    img = preprocess_input(path_to_tensor(img_path))
    prediction = np.argmax(ResNet50_model.predict(img))
    return ((prediction <= 268) & (prediction >= 151)) 

In [39]:
# load Dog data
train_files, train_targets = load_dataset(data_path+'dogImages/train')
valid_files, valid_targets = load_dataset(data_path+'dogImages/valid')
test_files, test_targets = load_dataset(data_path+'dogImages/test')

# load list of dog names
dog_names = [item[30:-1] for item in sorted(glob(data_path+"dogImages/train/*/"))]

In [40]:
# Preprocess Data
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|████████████████████████████████████████████████████████████████████████████████| 836/836 [00:08<00:00, 93.94it/s]


In [41]:
# A CNN Model use input directly
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=3, padding='valid', activation='relu', input_shape=train_tensors.shape[1:]))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=3, padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=3, padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(GlobalAveragePooling2D())
model.add(Dense(133, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 109, 109, 32)      4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 52, 52, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 26, 26, 64)        0         
_________________________________________________________________
global_average_pooling2d_4 ( (None, 64)                0         
__________

In [42]:
checkpointer = ModelCheckpoint(filepath='checkpoints/N2_7_weights.best.from_scratch.hdf5', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_tensors, train_targets, validation_data=(valid_tensors, valid_targets), epochs=10, batch_size=20, callbacks=[checkpointer], verbose=0)


Epoch 00001: val_loss improved from inf to 4.86950, saving model to checkpoints/N2_7_weights.best.from_scratch.hdf5

Epoch 00002: val_loss improved from 4.86950 to 4.83511, saving model to checkpoints/N2_7_weights.best.from_scratch.hdf5

Epoch 00003: val_loss improved from 4.83511 to 4.78148, saving model to checkpoints/N2_7_weights.best.from_scratch.hdf5

Epoch 00004: val_loss improved from 4.78148 to 4.76950, saving model to checkpoints/N2_7_weights.best.from_scratch.hdf5

Epoch 00005: val_loss improved from 4.76950 to 4.70687, saving model to checkpoints/N2_7_weights.best.from_scratch.hdf5

Epoch 00006: val_loss improved from 4.70687 to 4.70424, saving model to checkpoints/N2_7_weights.best.from_scratch.hdf5

Epoch 00007: val_loss improved from 4.70424 to 4.66218, saving model to checkpoints/N2_7_weights.best.from_scratch.hdf5

Epoch 00008: val_loss improved from 4.66218 to 4.62621, saving model to checkpoints/N2_7_weights.best.from_scratch.hdf5

Epoch 00009: val_loss improved from

In [43]:
model.load_weights('checkpoints/N2_7_weights.best.from_scratch.hdf5')
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 4.3062%


In [46]:
# use transfer learning, load bottleneck features VGG16
bottleneck_features = np.load('../ClassHelper/bottleneck_features/DogVGG16Data.npz')
train_VGG16 = bottleneck_features['train']
valid_VGG16 = bottleneck_features['valid']
test_VGG16 = bottleneck_features['test']

VGG16_model = Sequential()
VGG16_model.add(GlobalAveragePooling2D(input_shape=train_VGG16.shape[1:]))
VGG16_model.add(Dense(133, activation='softmax'))

VGG16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_5 ( (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 133)               68229     
Total params: 68,229
Trainable params: 68,229
Non-trainable params: 0
_________________________________________________________________


In [47]:
VGG16_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='checkpoints/N2_7_weights.best.VGG16.hdf5', verbose=1, save_best_only=True)

VGG16_model.fit(train_VGG16, train_targets, validation_data=(valid_VGG16, valid_targets), epochs=20, batch_size=20, callbacks=[checkpointer], verbose=0)


Epoch 00001: val_loss improved from inf to 10.35689, saving model to checkpoints/N2_7_weights.best.VGG16.hdf5

Epoch 00002: val_loss improved from 10.35689 to 9.84910, saving model to checkpoints/N2_7_weights.best.VGG16.hdf5

Epoch 00003: val_loss improved from 9.84910 to 9.41905, saving model to checkpoints/N2_7_weights.best.VGG16.hdf5

Epoch 00004: val_loss improved from 9.41905 to 9.13287, saving model to checkpoints/N2_7_weights.best.VGG16.hdf5

Epoch 00005: val_loss improved from 9.13287 to 9.05116, saving model to checkpoints/N2_7_weights.best.VGG16.hdf5

Epoch 00006: val_loss improved from 9.05116 to 8.88367, saving model to checkpoints/N2_7_weights.best.VGG16.hdf5

Epoch 00007: val_loss improved from 8.88367 to 8.88337, saving model to checkpoints/N2_7_weights.best.VGG16.hdf5

Epoch 00008: val_loss improved from 8.88337 to 8.66435, saving model to checkpoints/N2_7_weights.best.VGG16.hdf5

Epoch 00009: val_loss improved from 8.66435 to 8.26111, saving model to checkpoints/N2_7_

In [48]:
VGG16_model.load_weights('checkpoints/N2_7_weights.best.VGG16.hdf5')
VGG16_predictions = [np.argmax(VGG16_model.predict(np.expand_dims(feature, axis=0))) for feature in test_VGG16]

test_accuracy = 100*np.sum(np.array(VGG16_predictions)==np.argmax(test_targets, axis=1))/len(VGG16_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 47.1292%


In [49]:
# use transfer learning, load bottleneck features Xception
bottleneck_features = np.load('../ClassHelper/bottleneck_features/DogXceptionData.npz')
train_Xception = bottleneck_features['train']
valid_Xception = bottleneck_features['valid']
test_Xception = bottleneck_features['test']

Xception_model = Sequential()
Xception_model.add(GlobalAveragePooling2D(input_shape=train_Xception.shape[1:]))
Xception_model.add(Dense(256, activation='relu'))
Xception_model.add(Dropout(rate=0.3))
Xception_model.add(Dense(133, activation='softmax'))
Xception_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_6 ( (None, 2048)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               524544    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 133)               34181     
Total params: 558,725
Trainable params: 558,725
Non-trainable params: 0
_________________________________________________________________


In [51]:
Xception_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='checkpoints/N2_7_weights.best.Xception.hdf5', verbose=1, save_best_only=True)
Xception_model.fit(train_Xception, train_targets, validation_data=(valid_Xception, valid_targets), epochs=20, batch_size=100, callbacks=[checkpointer], verbose=0)


Epoch 00001: val_loss improved from inf to 0.52507, saving model to checkpoints/N2_7_weights.best.Xception.hdf5

Epoch 00002: val_loss improved from 0.52507 to 0.51640, saving model to checkpoints/N2_7_weights.best.Xception.hdf5

Epoch 00003: val_loss improved from 0.51640 to 0.51045, saving model to checkpoints/N2_7_weights.best.Xception.hdf5

Epoch 00004: val_loss did not improve from 0.51045

Epoch 00005: val_loss did not improve from 0.51045

Epoch 00006: val_loss did not improve from 0.51045

Epoch 00007: val_loss did not improve from 0.51045

Epoch 00008: val_loss did not improve from 0.51045

Epoch 00009: val_loss did not improve from 0.51045

Epoch 00010: val_loss did not improve from 0.51045

Epoch 00011: val_loss did not improve from 0.51045

Epoch 00012: val_loss did not improve from 0.51045

Epoch 00013: val_loss did not improve from 0.51045

Epoch 00014: val_loss did not improve from 0.51045

Epoch 00015: val_loss did not improve from 0.51045

Epoch 00016: val_loss did no

In [52]:
Xception_model.load_weights('checkpoints/N2_7_weights.best.Xception.hdf5')
Xception_predictions = [np.argmax(Xception_model.predict(np.expand_dims(feature, axis=0))) for feature in test_Xception]
test_accuracy = 100*np.sum(np.array(Xception_predictions)==np.argmax(test_targets, axis=1))/len(Xception_predictions)

print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 85.4067%


In [53]:
# single case prediction
def extract_Xception(tensor):
    from keras.applications.xception import Xception, preprocess_input
    return Xception(weights='imagenet', include_top=False).predict(preprocess_input(tensor))

def Xception_predict_breed(img_path):
    bottleneck_feature = extract_Xception(path_to_tensor(img_path))
    predicted_vector = Xception_model.predict(bottleneck_feature)
    return dog_names[np.argmax(predicted_vector)]

In [54]:
# dog detector
def my_detector(img_path):
    if dog_detector(img_path)>0:
        print("Hello, Dog! Your breed is likely...")
        print(Xception_predict_breed(img_path))
    elif face_detector(img_path)>0:
        print("Hello, Human! You look like a ...")
        print(Xception_predict_breed(img_path))
    else:
        print("Error, not human or dog")

my_detector('../ClassSampleData/1.jpg')

Hello, Dog! Your breed is likely...
rain\030.Border_terrier
